## OpenMMTools Integration

This enables OPS to use any testsystem from openmmtools to be used as well as add ANY possible OpenMM setup, not just the ones that we provided before. The old OpenMMEngine has been rename intp `SimpleOpenMMEngine`

In [1]:
from __future__ import print_function

import openpathsampling as paths

try:
    import openmm as omm
    import openmm.unit as u
except ImportError: # OpenMM < 7.6
    import simtk.openmm as omm
    import simtk.unit as u
import mdtraj as md

import openpathsampling.engines.openmm as eng

In [2]:
# NBVAL_IGNORE_OUTPUT
# Ignore until next OpenMMTools release (> 0.20.3), see openpathsampling/openpathsampling#1091
import openmmtools as omt

Create Alanine in Vacuum and run it using OPS.

In [3]:
testsystem = omt.testsystems.AlanineDipeptideVacuum()

Let's have a look at the content

In [4]:
# NBVAL_SKIP
{ key: type(value) for key, value in testsystem.__dict__.items()}

{'_system': openmm.openmm.System,
 '_positions': openmm.unit.quantity.Quantity,
 '_topology': openmm.app.topology.Topology,
 '_mdtraj_topology': NoneType}

An `OpenMM` simulation in OPS needs 3 ingredients to function
1. A template snapshot (which contains topolgy and coordinates/velocities), 
2. an OpenMM system object, and 
3. an OpenMM integrator object.

In the following we will get these parts.

#### 1. The template

The information for the template is contained in the `_topology` and the `_positions` object.

In [5]:
template = eng.snapshot_from_testsystem(testsystem)

#### 2. The system

This is easy since we directly get it from the testsytem objects

In [6]:
system = testsystem.system

#### 3. The integrator

The openmmtools system is (almost) independent of the integrator and `openmm` provides us with lots of options. For now we pick something simple.

In [7]:
integrator = omm.VerletIntegrator(
    0.002 * u.picoseconds    
)

build the engine

In [8]:
engine = eng.Engine(
    template.topology, 
    system, 
    integrator)

Let's run a simulation of 10 steps.

In [9]:
traj = engine.generate(template, [paths.LengthEnsemble(10).can_append])

In [10]:
assert len(traj) == 10

In [11]:
# NBVAL_SKIP
print(traj[5].coordinates)

[[ 0.18766724  0.11599682  0.01245003]
 [ 0.19458355  0.22444932  0.00401419]
 [ 0.1443672   0.27122667  0.08869705]
 [ 0.14886743  0.2574746  -0.08926158]
 [ 0.33994532  0.26638272  0.0060984 ]
 [ 0.42709577  0.18970107  0.04380053]
 [ 0.36886498  0.38998336 -0.03181566]
 [ 0.29336688  0.45243084 -0.05633819]
 [ 0.49859306  0.45657611 -0.02400118]
 [ 0.55814379  0.41515264  0.0573551 ]
 [ 0.57325351  0.43391329 -0.15561192]
 [ 0.51911271  0.48312774 -0.23640621]
 [ 0.67427629  0.47320485 -0.14414111]
 [ 0.58423072  0.32728142 -0.17536469]
 [ 0.47495225  0.60669041 -0.00127666]
 [ 0.359584    0.64924204  0.00474686]
 [ 0.58359265  0.68379164  0.00902625]
 [ 0.67474234  0.64066118  0.00332434]
 [ 0.57127434  0.82897234  0.01853087]
 [ 0.46718949  0.86095488  0.02347777]
 [ 0.62214309  0.86301541  0.108722  ]
 [ 0.61715913  0.87434274 -0.06931632]] nm


In [12]:
# NBVAL_IGNORE_OUTPUT
# covers mdtraj warning
psi = md.compute_psi(traj.to_mdtraj())

/home/sander/miniconda3/envs/python3/lib/python3.9/site-packages/mdtraj/utils/validation.py:115: TypeCastPerformanceWarning: Casting unitcell_vectors dtype=float64 to <class 'numpy.float32'> 
  warnings.warn("Casting %s dtype=%s to %s " % (name, val.dtype, dtype),


In [13]:
# NBVAL_IGNORE_OUTPUT
psi[1][3:8]

array([[ 3.1061459],
       [-3.0958638],
       [ 3.1367562],
       [ 3.1283274],
       [ 3.0931776]], dtype=float32)

In [14]:
st = paths.Storage('engine_store_test.nc', mode='w')

In [15]:
st.engines.save(engine);
st.tag['template'] = template

The engine is stored by using the XML serialization of OpenMM and put this into the storage.

In [16]:
# NBVAL_SKIP
st.variables['engines_json'][0][0:256] + '...'

'{"_cls":"OpenMMEngine","_dict":{"system_xml":"<?xml version=\\"1.0\\" ?>\\n<System openmmVersion=\\"7.6\\" type=\\"System\\" version=\\"1\\">\\n\\t<PeriodicBoxVectors>\\n\\t\\t<A x=\\"2\\" y=\\"0\\" z=\\"0\\"\\/>\\n\\t\\t<B x=\\"0\\" y=\\"2\\" z=\\"0\\"\\/>\\n\\t\\t<C x=\\"0\\" y=\\"0\\" z=\\"2...'

In [17]:
st.save(traj);

In [18]:
st.close()

### try to load the stored engine

In [19]:
st = paths.AnalysisStorage('engine_store_test.nc')

In [20]:
engine = st.engines[0]

Run another 10 steps to test

In [21]:
template = st.tag['template']

In [22]:
traj = engine.generate(template, [paths.LengthEnsemble(10).can_append])

In [23]:
print(traj)

Trajectory[10]


In [24]:
st.close()

And save the trajectory for future use (other tests)

In [25]:
st = paths.Storage('engine_store_test.nc', "a")
st.save(traj)
st.close()